# Unión de tablas de mejores shows y peliculas
•	Objetivo:
Clasificar las mejores peliculas y los mejores programas por genero.


In [ ]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 21.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pymongo

In [ ]:
# Datos en formato CSV
data_netflix = pd.read_csv("data_netflix.csv")
mejores_shows = pd.read_csv("Mejores_shows_netflix.csv")
mejores_peliculas = pd.read_csv("Mejores_películas_Netflix.csv")
actores = pd.read_csv("Actores.csv")

In [ ]:
# Conexión a MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Acceso a la BD
db = client["netflix"]

# Acceso a la colección
collection = db['shows']

In [ ]:
pipeline = [
   {
    $project: {
      MAIN_GENRE: 1,
      isShow: { $literal: true } # Se clasifica los documentos de shows como 'true'
    }
  },
  {
    $unionWith: {
      coll: "peliculas", # Unión con la colección peliculas
      pipeline: [
        {
          $project: {
            MAIN_GENRE: 1, # Sólo interesa el campo MAIN_GENRE
            isShow: { $literal: false } # Se clasifica los documentos de peliculas como 'false'
          }
        }
      ]
    }
  },
  {
    $group: {
      _id: "$MAIN_GENRE", // Se agrupa por MAIN_GENRE
      conteo_shows: {
        $sum: { $cond: [{ $eq: ["$isShow", true] }, 1, 0] } # Se cuentan solo los shows (isShow: true)
      },
      conteo_peliculas: {
        $sum: { $cond: [{ $eq: ["$isShow", false] }, 1, 0] } # Se cuentan solo las peliculas (isShow: false)
      }
    }
  },{
    $out: "genero_conteo"
  }
]

# Ejecutar la agregación
result = collection.aggregate(pipeline)
